In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path

In [ ]:
BASE_DIR = Path(__file__).parent.parent.parent.parent

In [ ]:
# --- CONFIG ---
MODEL_PATH = BASE_DIR / "models" / "wgisd-grape-cluster-detection-v1.pt"  # tu modelo YOLO entrenado
IMAGE_DIR = BASE_DIR / "data" / "yolo" / "images" / "val"
OUTPUT_CROP_DIR = BASE_DIR / "data" / "processed"
os.makedirs(OUTPUT_CROP_DIR, exist_ok=True)

In [3]:

# --- CARGAR MODELO ---
model = YOLO(MODEL_PATH)

In [7]:
# --- FUNCIONES AUXILIARES ---
def save_crop(img, box, output_path, idx):
    """Recorta el BB de la imagen y lo guarda."""
    x1, y1, x2, y2 = map(int, box)
    crop = img[y1:y2, x1:x2]
    cv2.imwrite(os.path.join(output_path, f"crop_{idx:03d}.jpg"), crop)
    return crop, (x1, y1, x2, y2)

def apply_mask_to_original(mask, box, original_shape):
    """
    Pega una máscara (binary mask) sobre la imagen original
    en la posición del bounding box.
    """
    x1, y1, x2, y2 = map(int, box)
    orig_mask = np.zeros((original_shape[0], original_shape[1]), dtype=np.uint8)
    # Redimensionar máscara al tamaño del BB original
    mask_resized = cv2.resize(mask, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
    orig_mask[y1:y2, x1:x2] = mask_resized
    return orig_mask

In [8]:
# --- PIPELINE ---
for img_name in os.listdir(IMAGE_DIR):
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(IMAGE_DIR, img_name)
    img = cv2.imread(img_path)
    
    results = model.predict(img, imgsz=640)  # retorna lista de Results
    results = results[0]  # si hay un solo frame

    boxes = results.boxes.xyxy.cpu().numpy()  # bounding boxes
    OUTPUT_IMG_DIR = os.path.join(OUTPUT_CROP_DIR, os.path.splitext(img_name)[0])
    os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

    for idx, box in enumerate(boxes):
        crop, box_coords = save_crop(img, box, OUTPUT_IMG_DIR, idx)

        # --- AQUÍ se aplicaría la segmentación de colores ---
        # Ejemplo: máscara dummy toda blanca (solo demostración)
        dummy_mask = np.ones(crop.shape[:2], dtype=np.uint8) * 255
        mask_on_orig = apply_mask_to_original(dummy_mask, box_coords, img.shape)

        # Opcional: guardar máscara sobre la imagen original
        mask_vis_path = os.path.join(OUTPUT_IMG_DIR, f"mask_on_orig_{idx:03d}.png")
        cv2.imwrite(mask_vis_path, mask_on_orig)

print("Recortes y máscaras iniciales generadas.")


0: 448x640 24 uva_bboxs, 29.7ms
Speed: 2.8ms preprocess, 29.7ms inference, 4.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 18 uva_bboxs, 4.4ms
Speed: 1.5ms preprocess, 4.4ms inference, 3.9ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 10 uva_bboxs, 4.3ms
Speed: 1.5ms preprocess, 4.3ms inference, 2.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 10 uva_bboxs, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 3.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 24 uva_bboxs, 4.3ms
Speed: 1.7ms preprocess, 4.3ms inference, 4.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 13 uva_bboxs, 4.6ms
Speed: 1.8ms preprocess, 4.6ms inference, 2.6ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 14 uva_bboxs, 4.5ms
Speed: 2.8ms preprocess, 4.5ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 19 uva_bboxs, 4.3ms
Speed: 2.0ms preprocess, 4.3ms inference, 4.2ms postprocess per image 